In [1]:
import cv2
import os

def create_video_from_frames(frames_dir, output_video_path, fps=30):
    # Get all frame filenames and sort them
    frame_files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.jpg') or f.endswith('.png')])
    
    if not frame_files:
        print(f"No frames found in {frames_dir}")
        return

    # Read the first frame to get the frame size
    frame = cv2.imread(os.path.join(frames_dir, frame_files[0]))
    height, width, layers = frame.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use other codecs like 'XVID'
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for frame_file in frame_files:
        frame = cv2.imread(os.path.join(frames_dir, frame_file))
        video.write(frame)

    # Release the video writer
    video.release()
    print(f"Video created at {output_video_path}")

# Base directory for test video frames
test_base_dir = '/kaggle/input/visdrone2019-mot-test/VisDrone2019-MOT-test-challenge/sequences'
output_base_dir = '/kaggle/working/test_videos'

# Ensure output directory exists
os.makedirs(output_base_dir, exist_ok=True)

# Loop through each video sequence directory and create a video
for video_dir in sorted(os.listdir(test_base_dir)):
    frames_dir = os.path.join(test_base_dir, video_dir)
    if os.path.isdir(frames_dir):  # Ensure it's a directory
        output_video_path = os.path.join(output_base_dir, f"{video_dir}.mp4")
        create_video_from_frames(frames_dir, output_video_path)
    else:
        print(f"{frames_dir} is not a directory. Skipping.")

print(f"All videos are created in {output_base_dir}")

Video created at /kaggle/working/test_videos/uav0000006_06900_v.mp4
Video created at /kaggle/working/test_videos/uav0000074_10080_v.mp4
Video created at /kaggle/working/test_videos/uav0000074_11856_v.mp4
Video created at /kaggle/working/test_videos/uav0000075_00000_v.mp4
Video created at /kaggle/working/test_videos/uav0000077_02880_v.mp4
Video created at /kaggle/working/test_videos/uav0000097_00000_v.mp4
Video created at /kaggle/working/test_videos/uav0000105_00001_v.mp4
Video created at /kaggle/working/test_videos/uav0000105_03338_v.mp4
Video created at /kaggle/working/test_videos/uav0000158_00000_v.mp4
Video created at /kaggle/working/test_videos/uav0000186_00250_v.mp4
Video created at /kaggle/working/test_videos/uav0000202_00000_v.mp4
Video created at /kaggle/working/test_videos/uav0000233_01200_v.mp4
Video created at /kaggle/working/test_videos/uav0000241_05232_v.mp4
Video created at /kaggle/working/test_videos/uav0000320_03910_v.mp4
Video created at /kaggle/working/test_videos/uav

In [2]:
pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.5/865.5 kB 24.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 3.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import numpy as np
import supervision as sv
from ultralytics import YOLO

# Load YOLO model and initialize tracker
model = YOLO("/kaggle/input/visdrone-yolov8x-weights/best.pt")
tracker = sv.ByteTrack()

# Initialize annotators
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

def process_video(video_path: str, output_path: str) -> None:
    def callback(frame: np.ndarray, _: int) -> np.ndarray:
        results = model(frame)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = tracker.update_with_detections(detections)

        labels = [
            f"#{tracker_id} {results.names[class_id]}"
            for class_id, tracker_id
            in zip(detections.class_id, detections.tracker_id)
        ]

        annotated_frame = box_annotator.annotate(frame.copy(), detections=detections)
        annotated_frame = label_annotator.annotate(annotated_frame, detections=detections, labels=labels)
        return trace_annotator.annotate(annotated_frame, detections=detections)

    # Process the video
    sv.process_video(
        source_path=video_path,
        target_path=output_path,
        callback=callback
    )

# Directories
input_dir = '/kaggle/working/test_videos/'
output_dir = '/kaggle/working/output_videos/'
os.makedirs(output_dir, exist_ok=True)

# Process each video in the input directory
for video_file in os.listdir(input_dir):
    if video_file.lower().endswith(('.mp4')):  # Adjust file types if necessary
        input_path = os.path.join(input_dir, video_file)
        output_path = os.path.join(output_dir, video_file)
        print(f"Processing {input_path}...")
        process_video(input_path, output_path)
        print(f"Saved processed video to {output_path}.")


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


Processing /kaggle/working/test_videos/uav0000202_00000_v.mp4...

0: 384x640 4 pedestrians, 16 peoples, 4 bicycles, 16 cars, 2 vans, 8 motors, 118.6ms
Speed: 10.5ms preprocess, 118.6ms inference, 182.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 pedestrians, 13 peoples, 4 bicycles, 14 cars, 1 van, 2 buss, 8 motors, 26.9ms
Speed: 3.1ms preprocess, 26.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 pedestrians, 13 peoples, 6 bicycles, 10 cars, 1 van, 1 awning-tricycle, 2 buss, 12 motors, 27.0ms
Speed: 1.9ms preprocess, 27.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 pedestrians, 12 peoples, 7 bicycles, 12 cars, 2 vans, 2 buss, 11 motors, 26.9ms
Speed: 2.2ms preprocess, 26.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 pedestrians, 15 peoples, 4 bicycles, 13 cars, 1 van, 2 buss, 11 motors, 27.0ms
Speed: 2.3ms preprocess, 27.0ms inference, 1.4ms postprocess per i

In [5]:
!rm -r /kaggle/working/test_videos